# import module

In [2]:
import json
import numpy as np
import pandas as pd
import math
import glob
import os
from tqdm import tqdm

# load data

In [ ]:
for file in glob.glob("../../etc/features/*.csv"):
    print(os.path.basename(file).split(".csv")[0])

# Benri Kansu

In [2]:
# 合成加速度
def get_SynAcc(x, y, z):
    return math.sqrt(x*x + y*y + z*z)

# 平均
def get_ave(df):
    stack_list=[]
    for row in df.itertuples():
        x, y, z=row[1], row[2], row[3]
        stack_list.append(get_SynAcc(x, y, z))
    return sum(stack_list)/len(stack_list)

# 各軸の平均
def get_Ave_value(value):
    return sum(value)/len(value)

# 各軸の標準偏差
def get_Std_value(value):
    stack_list=[]
    ave=get_Ave_value(value)
    stack_list=[(v-ave)**2 for v in value]
    return math.sqrt(sum(stack_list)/(len(value)-1))

# 幅
def get_Range(df):
    stack_list=[]
    for row in df.itertuples():
        x, y, z=row[1], row[2], row[3]
        stack_list.append(get_SynAcc(x, y, z))
    return max(stack_list) - min(stack_list)

# 標準偏差
def get_Std(df):
    stack_list=[]
    ave=get_ave(df)
    for row in df.itertuples():
        syn=get_SynAcc(row[1], row[2], row[3])
        stack_list.append((syn-ave)**2)
    return math.sqrt(sum(stack_list)/(len(df)-1))

# 歪度
def get_Skewness(df):
    stack_list=[]
    ave=get_ave(df)
    for row in df.itertuples():
        syn=get_SynAcc(row[1], row[2], row[3])
        stack_list.append((syn-ave)**3)
    return sum(stack_list)/(get_Std(df)**3)*(len(df)/((len(df)-1)*(len(df)-2)))

# 尖度
def get_Kurtosis(df):
    stack_list=[]
    ave=get_ave(df)
    for row in df.itertuples():
        syn=get_SynAcc(row[1], row[2], row[3])
        stack_list.append((syn-ave)**4)
    return ((sum(stack_list)/(get_Std(df)**4))*((len(df)*(len(df)+1))/((len(df)-1)*(len(df)-2)*(len(df)-3)))) - (3*(len(df)-1)**2)/((len(df)-2)*(len(df)-3))

# エネルギー
def get_Energy(df):
    stack_list=[]
    for row in df.itertuples():
        x, y, z=row[1], row[2], row[3]
        stack_list.append(get_SynAcc(x, y, z)**2)
    return sum(stack_list)

def get_Fft(df):
    stack_list=[]
    for row in df.itertuples():
        x,y,z=row[1], row[2], row[3]
        stack_list.append(get_SynAcc(x,y,z))
    return max(np.fft.fft(stack_list))

In [ ]:
df=pd.read_json("../../etc/no_header_pdr_raw_data/{}.json".format('5NM1shibataku'), orient='records', lines=True)
df=df[df['type']=='Accelerometer']

# タイムウインド重複なし

In [ ]:
df=pd.read_json("../../etc/no_header_pdr_raw_data/{}.json".format("7NM8miyazaki"), orient='records', lines=True)

In [ ]:
ROUND=1000

df_all=df[df['unixTime']//ROUND==((df['unixTime'].min()//ROUND)+30)]
df_acc=df_all[df_all['type']=='Accelerometer']

In [ ]:
stack_list=[]
for row in df_acc.itertuples():
    x,y,z=row[1], row[2], row[3]
    stack_list.append(get_SynAcc(x,y,z))

In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

def fft(time, ampl):
   
    # サンプリング周期[sec]の計算 #################################################
    sampling_cycle = time
   
    # データ数カウント ############################################################
    N = len(ampl)
   
    # 高速フーリエ変換（ FFT ） ####################################################
    fft_ampl = np.fft.fft(ampl)
   
    # FFT の複素数結果を絶対に変換 ###############################################
    abs_fft_amp = np.abs(fft_ampl)
   
    # 振幅をもとの信号に揃える #####################################################
    abs_fft_amp    = abs_fft_amp    / N * 2   # 交流成分
    abs_fft_amp[0] = abs_fft_amp[0] / 2       # 直流成分
   
    # 周波数軸のデータ作成 #######################################################
    frequency = np.linspace(0, 1.0/sampling_cycle, N) # 周波数軸　linspace(開始, 終了, 分割数)
   
    # ピーク検出 ################################################################
    maximal_idx = signal.argrelmax(abs_fft_amp[:int(N/2)+1], order=1) # 極大値インデックスの取得
    print(maximal_idx)
    print(abs_fft_amp)
   
    # グラフ表示 ################################################################
    plt.figure(figsize=(10, 8))
    plt.plot(frequency[:int(N/2)+1], abs_fft_amp[:int(N/2)+1])
    plt.scatter(frequency[maximal_idx], abs_fft_amp[maximal_idx], c='red', s=25)
    plt.grid(True)
    plt.title('Fast Fourier Transform')
    plt.xlabel('freqency[Hz]')
    plt.ylabel('amplitude')
    
    return fft_ampl
    
def low_pass_filter(fft_time, fft_amp, cut_off):
   
    # データ数カウント ############################################################
    N = len(fft_amp)
   
    # cut_off 以下の周波数の amplitude をゼロにする ################################
    cut_off2 = fft_time - cut_off
    fft_amp[((fft_time > cut_off)&(fft_time < cut_off2))] = 0 + 0j
          
    # グラフ用に実波形データに変換 #################################################
   
    # FFT の複素数結果を絶対に変換
    abs_fft_amp = np.abs(fft_amp)
   
    # 振幅をもとの信号に揃える
    abs_fft_amp    = abs_fft_amp    / N * 2 # 交流成分
    abs_fft_amp[0] = abs_fft_amp[0] / 2     # 直流成分
   
    # ピーク検出 ################################################################
    #maximal_idx = signal.argrelmax(abs_fft_amp[:int(N/2)+1], order=1) # 極大値インデックスの取得
    maximal_idx = signal.argrelmax(abs_fft_amp, order=1) # 極大値インデックスの取得
   
    # グラフ表示 ################################################################
    plt.figure(figsize=(10, 8))
    #plt.plot(fft_time[:int(N/2)+1], abs_fft_amp[:int(N/2)+1])
    #plt.scatter(fft_time[maximal_idx], abs_fft_amp[maximal_idx], c='red', s=25)
    plt.plot(fft_time, abs_fft_amp)
    plt.scatter(fft_time[maximal_idx], abs_fft_amp[maximal_idx], c='red', s=25)
    plt.grid(True)
    plt.title('Low Pass Filter')
    plt.xlabel('freqency[Hz]')
    plt.ylabel('amplitude')
   
    return fft_time, fft_amp

## 差分をとる

In [ ]:
ROUND=1000

for file in tqdm(glob.glob("../../etc/label/*.csv")):
    file_name=os.path.basename(file).split(".csv")[0]
    new_df=pd.DataFrame()
    label=pd.read_csv("../../etc/label/{}.csv".format(file_name), header=None)
    index=0
            
    df=pd.read_json("../../etc/no_header_pdr_raw_data/{}.json".format(file_name), orient='records', lines=True)

    print('{}LOAD DONE!!!!'.format(file_name))
            
    for i in range(label[0][0], label[0][len(label)-1]+1):
        df_all=df[df['unixTime']//ROUND==((df['unixTime'].min()//ROUND)+i)]
        df_acc=df_all[df_all['type']=='Accelerometer']
        df_gyro=df_all[df_all['type']=='Gyroscope']
        
        df_old_all=df[df['unixTime']//ROUND==((df['unixTime'].min()//ROUND)+i-1)]
        df_old_acc=df_old_all[df_old_all['type']=='Accelerometer']
        df_old_gyro=df_old_all[df_old_all['type']=='Gyroscope']
        if len(df_acc)==0 or len(df_gyro)==0:
            index+=1
            continue

        old_acc_x_ave, old_acc_y_ave, old_acc_z_ave, old_acc_range, old_acc_x_std, old_acc_y_std, old_acc_z_std, old_gyro_range, \
        old_gyro_x_ave, old_gyro_y_ave, old_gyro_z_ave, old_gyro_x_std, old_gyro_y_std, old_gyro_z_std, old_acc_std, old_gyro_std, \
        old_acc_skewness, old_gyro_skewness, old_acc_kurtosis, old_gyro_kurtosis, old_acc_energy, old_gyro_energy, old_acc_ave, old_gyro_ave= \
        get_Ave_value(df_old_acc['x']), get_Ave_value(df_old_acc['y']), get_Ave_value(df_old_acc['z']), get_Range(df_old_acc), \
        get_Std_value(df_old_acc['x']), get_Std_value(df_old_acc['y']), get_Std_value(df_old_acc['z']), get_Range(df_old_gyro), \
        get_Ave_value(df_old_gyro['x']), get_Ave_value(df_old_gyro['y']), get_Ave_value(df_old_gyro['z']), get_Std_value(df_old_gyro['x']), \
        get_Std_value(df_old_gyro['y']), get_Std_value(df_old_gyro['z']), get_Std(df_old_acc), get_Std(df_old_gyro), get_Skewness(df_old_acc), \
        get_Skewness(df_old_gyro), get_Kurtosis(df_old_acc), get_Kurtosis(df_old_gyro), get_Energy(df_old_acc), get_Energy(df_old_gyro), get_ave(df_old_acc), get_ave(df_old_gyro)

        acc_x_ave, acc_y_ave, acc_z_ave, acc_range, acc_x_std, acc_y_std, acc_z_std, gyro_range, \
        gyro_x_ave, gyro_y_ave, gyro_z_ave, gyro_x_std, gyro_y_std, gyro_z_std, acc_std, gyro_std, \
        acc_skewness, gyro_skewness, acc_kurtosis, gyro_kurtosis, acc_energy, gyro_energy, acc_ave, gyro_ave= \
        get_Ave_value(df_acc['x']), get_Ave_value(df_acc['y']), get_Ave_value(df_acc['z']), get_Range(df_acc), \
        get_Std_value(df_acc['x']), get_Std_value(df_acc['y']), get_Std_value(df_acc['z']), get_Range(df_gyro), \
        get_Ave_value(df_gyro['x']), get_Ave_value(df_gyro['y']), get_Ave_value(df_gyro['z']), get_Std_value(df_gyro['x']), \
        get_Std_value(df_gyro['y']), get_Std_value(df_gyro['z']), get_Std(df_acc), get_Std(df_gyro), get_Skewness(df_acc), \
        get_Skewness(df_gyro), get_Kurtosis(df_acc), get_Kurtosis(df_gyro), get_Energy(df_acc), get_Energy(df_gyro), get_ave(df_acc), get_ave(df_gyro)

        new_df=new_df.append({'label':label[1][index], 'user':file_name, 'acc_x_ave':acc_x_ave, 'acc_y_ave':acc_y_ave, 'acc_z_ave':acc_z_ave, 
                              'acc_range':acc_range, 'acc_x_std':acc_x_std, 'acc_y_std':acc_y_std, 'acc_z_std':acc_z_std, 'gyro_range':gyro_range, 
                              'gyro_x_ave':gyro_x_ave, 'gyro_y_ave':gyro_y_ave, 'gyro_z_ave':gyro_z_ave, 'gyro_x_std':gyro_x_std, 
                              'gyro_y_std':gyro_y_std, 'gyro_z_std':gyro_z_std,'acc_std':acc_std, 'gyro_std':gyro_std, 'acc_skewness':acc_skewness, 
                              'gyro_skewness':gyro_skewness, 'acc_kurtosis':acc_kurtosis, 'gyro_kurtosis':gyro_kurtosis, 'acc_energy':acc_energy, 
                              'gyro_energy':gyro_energy, 'acc_ave':acc_ave, 'gyro_ave':gyro_ave, 
                              'dif_acc_x_ave':(acc_x_ave-old_acc_x_ave), 'dif_acc_y_ave':(acc_y_ave-old_acc_y_ave), 'dif_acc_z_ave':(acc_z_ave-old_acc_z_ave), 
                              'dif_acc_range':(acc_range-old_acc_range), 'dif_acc_x_std':(acc_x_std-old_acc_x_std), 
                              'dif_acc_y_std':(acc_y_std-old_acc_y_std), 'dif_acc_z_std':(acc_z_std-old_acc_z_std), 
                              'dif_gyro_range':(gyro_range-old_gyro_range), 'dif_gyro_x_ave':(gyro_x_ave-old_gyro_x_ave), 'dif_gyro_y_ave':(gyro_y_ave-old_gyro_y_ave), 
                              'dif_gyro_z_ave':(gyro_z_ave-old_gyro_z_ave), 'dif_gyro_x_std':(gyro_x_std-old_gyro_x_std), 
                              'dif_gyro_y_std':(gyro_y_std-old_gyro_y_std), 'dif_gyro_z_std':(gyro_z_std-old_gyro_z_std), 'dif_acc_std':(acc_std-old_acc_std), 
                              'dif_gyro_std':(gyro_std-old_gyro_std), 'dif_acc_skewness':(acc_skewness-old_acc_skewness), 
                              'dif_gyro_skewness':(gyro_skewness-old_gyro_skewness), 'dif_acc_kurtosis':(acc_kurtosis-old_acc_kurtosis), 
                              'dif_gyro_kurtosis':(gyro_kurtosis-old_gyro_kurtosis), 'dif_acc_energy':(acc_energy-old_acc_energy), 
                              'dif_gyro_energy':(gyro_energy-old_gyro_energy), 'dif_acc_ave':(acc_ave-old_acc_ave), 'dif_gyro_ave':(gyro_ave-old_gyro_ave)}, 
                         ignore_index=True)

        index+=1
    new_df.to_csv("../../etc/dif_features/{}.csv".format(file_name), index=False)
    print('{}DONE!!!!'.format(file_name))

# タイムウインドウ重複

In [ ]:
glob.glob('../../etc/label/*.csv')[66]

In [ ]:
for file in tqdm(glob.glob("../../etc/label/*.csv")):
    file_name=os.path.basename(file).split(".csv")[0]
    all_df=pd.DataFrame()
    label=pd.read_csv("../../etc/label/{}.csv".format(file_name), header=None)
    index=0
            
    df=pd.read_json("../../etc/no_header_pdr_raw_data/{}.json".format(file_name), orient='records', lines=True)

    print('{}LOAD DONE!!!!'.format(file_name))
    
    for i in range(label[0][0], label[0][len(label)-1]+1):
        new_df=pd.DataFrame()
        new_df=new_df.append({'label':label[1][index], 'user':file_name}, ignore_index=True)
        for window in range(0, 4):
            features=get_features(df, i, window)
            if len(features)==0:
                index+=1
                continue
            new_df=pd.concat([new_df, features], axis=1)
        all_df=all_df.append(new_df)
            
        index+=1
    all_df.to_csv("../../etc/windows_features/{}.csv".format(file_name), index=False)
    print('{}DONE!!!!'.format(file_name))

In [ ]:
ROUND=1000

for file in glob.glob("../../etc/label/*.csv")[1:]:
    file_name=os.path.basename(file).split(".csv")[0]
    df=pd.DataFrame(columns=["x", "y", "z", "unixTime", "type"])
    new_df=pd.DataFrame(columns=["label", "acc_range", "acc_std", "acc_skewness", "acc_kurtosis", "acc_energy"])
    label=pd.read_csv("../../etc/label/{}.csv".format(file_name), header=None)
    index=0
    is_first=True

    with open("../../etc/pdr_raw_data/{}.json".format(file_name), "r") as f:
        lines=f.readlines()
        for line in lines[1:]:
            l=json.loads(line)
            df=df.append({'x':l["x"], 'y':l["y"], 'z':l["z"], 'unixTime':l["unixTime"], 'type':l["type"]}, ignore_index=True)

    print('{}LOAD DONE!!!!'.format(file_name))
    
    for i in range(label[0][0], label[0][len(label)-1]+1):
        if is_first==True:
            df_acc=df[df['unixTime']//ROUND==((df['unixTime'].min()//ROUND)+i)]
            df_acc=df_acc[df_acc['type']=='Accelerometer']
            is_first=False
        else:
            df_acc=df[((df['unixTime']//ROUND)-1)==((df['unixTime'].min()//ROUND)+i-1)]
            df_acc=df_acc.append(df[df['unixTime']//ROUND==((df['unixTime'].min()//ROUND)+i)], ignore_index=True)
            df_acc=df_acc[df_acc['type']=='Accelerometer']
        if len(df_acc)==0:
            index+=1
            continue
        new_df=new_df.append({'label':label[1][index], 'acc_range':get_Range(df_acc), 'acc_std':get_Std(df_acc), 
                              'acc_skewness':get_Skewness(df_acc), 'acc_kurtosis':get_Kurtosis(df_acc), 'acc_energy':get_Energy(df_acc)}, 
                             ignore_index=True)
        index+=1
    new_df.to_csv("../../etc/timed_features/{}.csv".format(file_name), index=False)
    print('{}DONE!!!!'.format(file_name))

# raw_dataへのラベル付け

In [ ]:
ROUND=1000

for file in glob.glob("../../etc/label/*.csv"):
    file_name=os.path.basename(file).split(".csv")[0]
    new_df=pd.DataFrame()
    label=pd.read_csv("../../etc/label/{}.csv".format(file_name), header=None)
    index=0
    
    df=pd.read_json("../../etc/no_header_pdr_raw_data/{}.json".format(file_name), orient='records', lines=True)

    print('{}LOAD DONE!!!!'.format(file_name))
            
    for i in range(label[0][0], label[0][len(label)-1]+1):
        df_acc=df[df['unixTime']//ROUND==((df['unixTime'].min()//ROUND)+i)]
        # df_acc=df_acc[df_acc['type']=='Accelerometer']
        if len(df_acc)==0:
            index+=1
            continue
        df_acc['user']=file_name
        df_acc['label']=label[1][index]
        new_df=new_df.append(df_acc, ignore_index=True)
        index+=1
    new_df.to_csv("../../etc/labeled_pdr_raw_data/{}.csv".format(file_name), index=False)
    print('{}DONE!!!!'.format(file_name))

# データ作るぞ

In [ ]:
for file in glob.glob("../../etc/test/*.csv"):
    file_name=os.path.basename(file).split(".csv")[0]
    df_features=pd.read_csv("../../etc/test/{}.csv".format(file_name))
    df_features['user']=file_name
    df_features.to_csv("../../etc/test/{}.csv".format(file_name), index=False)

In [ ]:
# awk 'NR==1 || FNR!=1' *.csv

# clipping

In [3]:
df=pd.read_csv('../../etc/step_num/features.csv')
print(len(df))
dfs=df.drop(['label', 'user'], axis=1)

for column in dfs.columns:
    p01=dfs[column].quantile(0.01)
    p99=dfs[column].quantile(0.99)
    dfs[column]=dfs[column].clip(p01, p99)
print(len(dfs))
# print(max(dfs['acc_ave']))

29360
29360


In [4]:
dfs['user']=df['user']
dfs['label']=df['label']

In [5]:
df.to_csv('../../etc/step_num/clipped_raw_features.csv', index=False)

# 正規化

In [6]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import scipy.stats as stats

In [7]:
df_features=pd.read_csv('../../etc/step_num/clipped_raw_features.csv')

In [8]:
df_features=df_features.dropna()

In [9]:
df_features.columns

Index(['SC', 'label', 'user'], dtype='object')

In [10]:
columns=df_features.columns[:1]

In [11]:
sc = StandardScaler()

for column in columns:
    # print(stats.zscore(df_features[column].values))
    df_features[column]=stats.zscore(df_features[column].values)

In [17]:
df_features.to_csv('../../etc/step_num/clipped_std_features.csv', index=False)

In [ ]:
minmax = MinMaxScaler(feature_range=(-1, 1))
mm_acc=minmax.fit_transform(x_train)

In [ ]:
new_df=pd.DataFrame()

for row in df_features.itertuples():
    new_df=new_df.append({'label':row[14], 'user':row[15], 'acc_range':std_acc[row[0]][3], 'acc_std':std_acc[row[0]][5], 
                          'acc_skewness':std_acc[row[0]][4], 'acc_kurtosis':std_acc[row[0]][2], 'acc_energy':std_acc[row[0]][1], 
                          'gyro_range':std_acc[row[0]][9], 'gyro_std':std_acc[row[0]][11], 'gyro_skewness':std_acc[row[0]][10], 
                          'gyro_kurtosis':std_acc[row[0]][8], 'gyro_energy':std_acc[row[0]][7], 'acc_ave':std_acc[row[0]][0], 
                          'gyro_ave':std_acc[row[0]][6]},  
                             ignore_index=True)

In [ ]:
new_df.to_csv('../../etc/labeled_features/clipped_std_features.csv', index=False)

# 時刻t-1のラベルを追加

In [ ]:
df_features=pd.read_csv('../../etc/dif_features/clipped_std_features.csv')

In [ ]:
labels=[]
new_df=pd.DataFrame()

for file in glob.glob("../../etc/label/*.csv"):
    labels=[]
    file_name=os.path.basename(file).split(".csv")[0]
    df=df_features[df_features['user']==file_name]
    labels+=[list(df['label'])[0]]
    labels+=list(df['label'])[:len(df)-1]
    df['pre_label']=labels
    new_df=new_df.append(df)

In [ ]:
new_df.to_csv('../../etc/dif_features/clipped_std_prelabel_features.csv', index=False)

# PCA

In [ ]:
df=pd.read_csv('../../etc/windows_features/clipped_std_features.csv')

In [ ]:
features=df.drop(['label', 'user'], axis=1)

In [ ]:
pca.fit(features)
features = pca.transform(features)

In [ ]:
pd.DataFrame(features, columns=["PC{}".format(x + 1) for x in range(len(df.drop(['label', 'user'], axis=1).columns))]).head()

In [ ]:
pd.DataFrame(features, columns=["PC{}".format(x + 1) for x in range(len(df.drop(['label', 'user'], axis=1).columns))]).head()

In [ ]:
# 累積寄与率を図示する
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
plt.gca().get_xaxis().set_major_locator(ticker.MaxNLocator(integer=True))
plt.plot([0] + list( np.cumsum(pca.explained_variance_ratio_)), "-o")
plt.xlabel("Number of principal components")
plt.ylabel("Cumulative contribution rate")
plt.grid()
plt.show()

In [ ]:
new_df=pd.DataFrame(features, columns=["PC{}".format(x + 1) for x in range(len(df.drop(['label', 'user'], axis=1).columns))])

In [ ]:
new_df=new_df.iloc[:, 0:16]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
# 主成分数を指定して、PCA のインスタンスを生成
pca = PCA()
# ロジスティック回帰のインスタンスを生成
lr = LogisticRegression()
# トレーニングデータとテストデータで PCA を実行
# X_train_pca = pca.fit_transform(features)
# X_test_pca = pca.transform(X_test_std)

In [ ]:
new_df.to_csv('../../etc/windows_features/pca.csv', index=False)